## Introduction

Ce programme a pour but de tester la plausibilité de la prononciation d'un mot telle qu'indiquée sur le Wiktionnaire.

Pour ce faire, il parcours itérativement les différents sons (phonemes) d'un mot et essaie les différentes lettres (graphemes) pouvant être utilisées pour écrire le son (grâce à une table de correspondance similaire à https://fr.wiktionary.org/wiki/Annexe:Prononciation/français#Troisième_approche). A chaque essai, il compare ce qu'il obtient avec l'orthographe utilisée dans le Wiktionnaire. Lorsqu'il arrive au même résultat, la prononciation du mot est jugée plausible. A l'inverse, s'il n'arrive pas à transcrire la même orthographe que celle indiquée dans le Wiktionnaire, la prononciation est jugée suspecte.

Ce programme peut aussi compter les différentes lettres utilisées pour transcrire un son. Ainsi, s'il est executé sur tous les mots du Wiktionnaire, il peut servir à compter les probabilités de ces lettres pour transcrire un son donné.



## Données d'entrée

In [1]:
# importation des librairies tierces
import datetime
import pandas as pd
from pathlib import Path

def add_words(df, mot, prononciation, type='à-faire'):
  df = df.append(dict(zip(['Mot', 'Prononciation', 'H_aspiré', 'Type', 'Pré_valide', 'Err_Code', 'Err_Label'],
                          [mot, prononciation, 0.0, type, 1.0, '-', '-'])), ignore_index=True)
  return df

# lecture des données d'entrée (ligne contenant un mot du dictionnaire
# ainsi que sa prononciation
def read_df():

  '''Read the input data (i.e. lines containing a Wiktionary word 
  and its pronunciation) '''
  filename = 'fr_wiktionary_full.csv'
    
  filepath = Path(filename)

  #url = 'https://fonétik.fr/'
  #url_filename = url + filename

  #print('downloading ', url_filename)
  # if file not found locally, then download it
  #if not filepath.exists():
  #!wget -N -q {url_filename}

  df = pd.read_csv(filename, keep_default_na=False, sep = '\t')

  #Ajout de mots absents du fichier d'input à cause de leur valeur
  #en conflit avec une lirairie python
  missing_words = [
                   ('naan', 'nan', 'nom'),
                   ('naans', 'nan', 'nom_flexion'),
                   ('na', 'na', 'particule'),
                   ('na', 'na', 'interjection'),
  ]
  #for w in missing_words:
  #  df = add_words(df, w[0], w[1], w[2])
  
  # Ajout de certains prononciations de mots non-présentes dans les entrées
  # (manque la 2ème prononciation dans le csv)
  temporary_words  = [#('argh', 'aʁ'),
                      ('bobsleigh', 'bɔbslɛ'),
                      ('bobsleighs', 'bɔbslɛ'),
                      ('os', 'o'),
                      #('résumpte', 'ʁezɔ̃t'),
                      #('exact', 'ɛɡza'),
                      #('exacts', 'ɛɡza'),
                       # should be in next version of csv
                      ('trapp', 'tʁap'),
                      ('trapps', 'tʁap'),
                      ('judd', 'ʒud'),
                      ('judds', 'ʒud'),
                      ('jodhs', 'jɔd'),
                      ('yachtsman', 'jɔtman'),
                      ('gonzs','ɡɔ̃z'),
                      ('hés','e'),
                      ('eins','ɛ̃'),
                      ('aveuent','avœ'),
                      ('hient','i'),
                      ('huent','y'),
                      ('hues','y'),
                      ('eaux-fortes','ofɔʁt'),
                      ('hauts-fonds','ofɔ̃'),
                      ('hoax','oks'),
                      ('curaçao','kyʁaso'),
                      ('curaçaos','kyʁaso'),
                      ('wisigoths','viziɡɔ'),
                      ('wisigoth','viziɡo'),
                      ('wisigoths','viziɡo'),
                      ('hon','ɔ̃'),
                      ('exempts','ɛɡzɑ̃'),
                      ('ah','a'),
                      ('s’', 's'),

  ]
  for temporary_word in temporary_words:
    df = add_words(df, temporary_word[0], temporary_word[1])

  return df

df = read_df()
df.rename(columns = {'Err_Code':'Warn_Code', 'Err_Label':'Warn_Label'}, inplace = True) 

In [2]:
df.tail()

,Mot,Prononciation,H_aspiré,Type,Pré_valide,Warn_Code,Warn_Label
1484038,wisigoths,viziɡo,0.0,à-faire,1.0,-,-
1484039,hon,ɔ̃,0.0,à-faire,1.0,-,-
1484040,exempts,ɛɡzɑ̃,0.0,à-faire,1.0,-,-
1484041,ah,a,0.0,à-faire,1.0,-,-
1484042,s’,s,0.0,à-faire,1.0,-,-


In [3]:
# corriger les & mal-formatés s'il y a en
df['Mot'] = df['Mot'].str.replace('&amp;', '&')

In [4]:
# Affichage du nombre d'échantillons en entrées
df.shape[0]

1484043

In [5]:
df['Plausible'] = '?'

In [6]:
# Affichage de 5 derniers échantillons
df.sample(5)

,Mot,Prononciation,H_aspiré,Type,Pré_valide,Warn_Code,Warn_Label,Plausible
756789,macellâtes,ma.sɛ.lat,0.0,verbe_flexion,1.0,-,-,?
760627,majusculisons,ma.ʒys.ky.li.zɔ̃,0.0,verbe_flexion,1.0,-,-,?
1061158,regalérasse,ʁə.ɡa.le.ʁas,0.0,verbe_flexion,1.0,-,-,?
132585,auto-démolissaient,o.to.de.mɔ.li.sɛ,0.0,verbe_flexion,1.0,-,-,?
1302625,sous-indexés,su.zɛ̃.dɛk.se,0.0,verbe_flexion,1.0,-,-,?


In [7]:
df[df.Mot=='résumpte']

,Mot,Prononciation,H_aspiré,Type,Pré_valide,Warn_Code,Warn_Label,Plausible
1248966,résumpte,ʁe.zɔ̃pt,0.0,nom,1.0,-,-,?


## Table de correspondance entre sons et lettres



In [8]:
# Table globale indiquant pour chaque phonème, les graphèmes possibles selon 
# la position du phonème dans le mot
phonemes2graphemes = {
    'm' : {
        'isolé' : {
            "m’" : {},
        },
        'début' : {
            'hm' : {},
            'm' : {},
            #'mh' : {},
        },
        'milieu' : {
            'm' : {},
            'me' : {},
            #'mh' : {},
            'mm' : {},
            'mme' : {},
        },
        'fin' : {
            'm' : {},
            'me' : {},
            'ment' : {},
            'mes' : {},
            'mm' : {},
            'mme' : {},
            'mment' : {},
            'mmes' : {},
            'mms' : {},
            'ms' : {},
        },
    },
    'v' : {
        'isolé' : {
        },
        'début' : {
            'v' : {},
            'w' : {},
        },
        'milieu' : {
            'bv' : {},
            'v' : {},
            've' : {}, ## écodéveloppements
            'w' : {},
            #'ʋ' : {},
        },
        'fin' : {
            'f' : {},
            'v' : {},
            've' : {},
            'vent' : {},
            'ves' : {},
            'vs' : {}, ## youvs
            'we' : {},
            'went' : {},
            'wes' : {},
        },
    },
    'f' : {
        'isolé' : {
        },
        'début' : {
            'f' : {},
            'ph' : {},
        },
        'milieu' : {
            'f' : {},
            'fe' : {},
            'ff' : {},
            'ffe' : {},
            'pf' : {},
            'ph' : {},
        },
        'fin' : {
            'f' : {},
            'fe' : {},
            'ff' : {}, ## apérispliff
            'fent' : {},
            'fes' : {},
            'ffe' : {},
            'ffent' : {},
            'ffes' : {},
            'fs' : {},
            'gh' : {},
            'pf' : {},
            'ph' : {},
            'phe' : {},
            'phent' : {},
            'phes' : {},
            'phs' : {},
            'v' : {},
            've' : {},
            'vs' : {},
        },
    },
      'b' : {
        'isolé' : {
        },
        'début' : {
            'b' : {},
            'bh' : {},
            'gb' : {}, ## gbagbiste (gabon)
        },
        'milieu' : {
            'b' : {},
            'be' : {},
            'bb' : {},
            'bbe' : {},
            'h' : {}, ## kahba
        },
        'fin' : {
            'b' : {},
            'bbe' : {},
            'bbent' : {},
            'bbes' : {},
            'be' : {},
            'bent' : {},
            'bes' : {},
            'bs' : {},
        },
    },
    'p' : {
        'isolé' : {
        },
        'début' : {
            'p' : {},
            #'ph' : {},
        },
        'milieu' : {
            'b' : {},
            'p' : {},
            'pe' : {},
            'pp' : {},
            'ps' : {}, ##corpsard
            'ppe' : {},
        },
        'fin' : {
            'p' : {},
            'pe' : {},
            'pent' : {},
            'pes' : {},
            'pp' : {},
            'ppe' : {},
            'ppent' : {},
            'ppes' : {},
            'pps' : {},
            'ps' : {},
        },
    },
    'd' : {
        'isolé' : {
            "d’" : {},
        },
        'début' : {
            'd' : {},
            'dh' : {},
            'j' : {}, ## for jaïnas
        },
        'milieu' : {
            'd' : {},
            'dd' : {},
            'de' : {},
            'dh' : {},
            'g' : {}, ## adagio, 
            'j' : {}, ## homejacker
            'z' : {},
        },
        'fin' : {
            'd' : {},
            'dd' : {},
            'dde' : {},
            'ddes' : {},
            'ddent' : {},
            'de' : {},
            'dent' : {},
            'des' : {},
            'dds' : {},
            'dh' : {},
            'dhs' : {},
            'ds' : {},
        },
    },
    't' : {
        'isolé' : {
            "t’" : {},
        },
        'début' : {
            'c' : {}, ## chatter
            't' : {},
            'th' : {},
        },
        'milieu' : {
            'c' : {}, ## rechatter
            'cht' : {},
            'chts' : {},
            'd' : {},
            'pt' : {},
            't' : {},
            'te' : {},
            'th' : {},
            'the' : {},
            'tt' : {},
            'tte' : {},
        },
        'fin' : {
            'cht' : {},
            'chts' : {},
            'd' : {},
            'dt' : {},
            'pt' : {},
            't' : {},
            'te' : {},
            'tent' : {},
            'tes' : {},
            'th' : {},
            'the' : {},
            'thent' : {},
            'thes' : {},
            'ths' : {},
            'ts' : {},
            'tt' : {},
            'tte' : {},
            'ttent' : {},
            'ttes' : {},
            'tts' : {},
        },
    },
    'n' : {
        'isolé' : {
            "n’" : {},
        },
        'début' : {
            'n' : {},
        },
        'milieu' : {
            'g' : {}, ## bignoler
            'hn' : {}, ## acehnais
            'mn' : {},
            'n' : {},
            'ñ' : {}, ## buñueliser
            'ne' : {},
            'nh' : {},
            'nn' : {},
            'nne' : {},
            'wn': {}, ## downloader
        },
        'fin' : {
            'mne' : {},
            'mnent' : {},
            'mnes' : {},
            'n' : {},
            'nh' : {},
            'nhs' : {},
            'ne' : {},
            'nent' : {},
            'nes' : {},
            'nn' : {},
            'nne' : {},
            'nnent' : {},
            'nnes' : {}, 
            'nns' : {},
            'ns' : {},
        },
    },
    'ɲ' : {
        'isolé' : {
        },
        'début' : {
            'gn' : {},
            'n' : {},
            'ñ' : {},
        },
        'milieu' : {
            'gn' : {},
            'gne' : {},
            'gni' : {}, ## torgnioler
            'ign' : {},
            'ñ' : {},
            'ni' : {}, ## orignal
            'nni' : {}, ## aiguillonniers
            'ny' : {},
        },
        'fin' : {
            'gne' : {},
            'gnent' : {},
            'gnes' : {},
            'igne' : {},
            'ny': {},  ## yidiny
            'nys': {}, ## yidinys
        },
    },
    'ŋ' : {
        'isolé' : {
        },
        'début' : {
            'n' : {},
        },
        'milieu' : {
            'n' : {},
            'ng' : {},
        },
        'fin' : {
            'g' : {}, ##hmong
            'ng' : {},
            'ngs' : {},
        },
    },
    'ɡ' : {
        'isolé' : {
        },
        'début' : {
            'g' : {},
            'gh' : {},
            'gu' : {},
        },
        'milieu' : {
            'c' : {},
            'g' : {},
            'gh' : {},
            'gg' : {},
            'gu' : {},
            'gue' : {},
        },
        'fin' : {
            'c' : {}, ## zinc
            'g' : {}, ## agreg
            'gge' : {},
            'ggent' : {},
            'gges' : {},
            #'ggs' : {}, ##briggs (not in Wix)
            'gs' : {},
            'gue' : {},
            'guent' : {},
            'gues' : {},
        },
    },
    'k' : {
        'isolé' : {
            "qu’" : {},
        },
        'début' : {
            'c' : {},
            'ch' : {},
            'k' : {},
            'kh' : {},
            'q' : {},
            'qu' : {}, ## questionneras
            'xh' : {},
        },
        'milieu' : {
            'c' : {},
            'cc' : {},
            'cch' : {},
            'ch' : {},
            'cg' : {}, ## brunschvicgien
            'ck' : {},
            #'ckh' : {},
            'cq' : {}, ## acquaintance 
            'cqu' : {},
            'cque' : {},
            'g' : {},
            'k' : {},
            'kh' : {},
            'kk' : {},
            'kw' : {}, ## atikamekwiser
            'q' : {},
            'qu' : {},
            'que' : {},
        },
        'fin' : {
            'c' : {},
            'ch' : {},
            'che' : {}, ## acéphalobrache
            'ches': {}, ## acritarches
            'chs' : {},
            'ck' : {},
            'cke' : {},
            'ckent' : {},
            'ckes' : {},
            'cks' : {},
            'cq' : {},
            'cque' : {},
            'cquent' : {},
            'cques' : {},
            'cs' : {},
            'g' : {},
            'k' : {},
            'ke' : {},
            'kent' : {},
            'kes' : {},
            'kh' : {},
            'khe' : {},
            'khs' : {},
            'khes' : {},
            'kke' : {},
            'kkent' : {},
            'kkes' : {},
            'ks' : {},
            'kw' : {}, ## atikamekw
            'kws' : {},
            'q' : {},
            'qs' : {},
            'que' : {},
            'quent' : {},
            'ques' : {},
        },
    },
    'ʒ' : {
        'isolé' : {
            "j’" : {},
        },
        'début' : {
            'g' : {},
            'ge' : {},
            'j' : {},
        },
        'milieu' : {
            'e' : {}, ## challengea
            'g' : {},
            'ge' : {},
            'gg' : {}, ## arpeggione
            'j' : {},
            'je' : {},
        },
        'fin' : {
            'g' : {},
            'ge' : {}, ## for voltage
            'gent' : {},
            'ges' : {},
            'j' : {},
            'je' : {},
            'jent' : {},
            'jes' : {},
            'js' : {},
        },
    },
    'ʃ' : {
        'isolé' : {
        },
        'début' : {
            'ch' : {},
            'ch’' : {}, ##ch’ti
            'che' : {},
            #!'h' : {}, ##chatter
            's' : {},
            'sch' : {},
            'sh' : {},
            'x' : {},
            'zh' : {},
        },
        'milieu' : {
            'c' : {}, ##acciaccatura
            'ch' : {},
            'che' : {},
            'chs' : {},
            'h' : {}, ##chatter
            's' : {},
            'sc' : {},
            'sch' : {},
            'sh' : {},
            'x' : {},
            'z' : {},##czamara
        },
        'fin' : {
            'ch' : {},
            'che' : {},
            'chent' : {},
            'ches' : {},
            'chs' : {},
            'sch' : {},
            'sche' : {},
            'schent' : {},
            'sches' : {},
            'schs' : {},
            'sh' : {},
            'she' : {},
            'shes' : {},
            'shent' : {},
            'shs' : {},
        },
    },
    'z' : {
        'isolé' : {
        },
        'début' : {
            'j' : {},
            'z' : {},
        },
        'milieu' : {
            's' : {},
            'se' : {},
            'si' : {},
            'x' : {},
            'z' : {},
            'zh' : {}, ##alzheimer
            'ze' : {},
            'zz' : {},
        },
        'fin' : {
            's' : {},
            'se' : {},
            'sent' : {},
            'ses' : {},
            'x' : {},
            'z' : {},
            'ze' : {},
            'zent' : {},
            'zes' : {},
            'zh' : {},
            'zs' : {},
            'zz' : {},
            'zze' : {},
            'zzent' : {},
            'zzes' : {},
        },
    },
    's' : {
       'isolé' : {
            "c’" : {},
            "s’" : {},
            "ç’" : {},
        },
        'début' : {
            'c' : {},
            'ç' : {},
            's' : {},
            'sc' : {},
            'th' : {},
            'x' : {},
        },
        'milieu' : {
            'c' : {},
            'cc' : {}, ##accenser
            'ç' : {},
            'ce' : {},
            's' : {},
            'sc' : {},
            'sç' : {},
            'se' : {},
            'ss' : {},
            'sse' : {},
            'sth' : {},
            't' : {},
            'x' : {},
            'z' : {},
        },
        'fin' : {
            'ce' : {},
            'cent' : {},
            'ces' : {},
            #'e' : {}, ##luxe (because of final e)
            's' : {},
            'sce' : {},
            'scent' : {},
            'sces' : {},
            'ss' : {},
            'se' : {},
            'sent' : {},
            'ses' : {},
            'sse' : {},
            'ssent' : {},
            'sses' : {},
            'th' : {},
            'ths' : {},
            'tz' : {},
            'x' : {},
            'z' : {},
        },
    },
    'ʁ' : {
        'isolé' : {
        },
        'début' : {
            #'hr' : {}, #Hrant
            'r' : {},
            'rh' : {},
            'rr' : {},
        },
        'milieu' : {
            'r' : {},
            'r’' : {}, ##entr’assassiner
            'rc' : {}, ##antarctique
            're' : {},
            'rh' : {},
            'rr' : {},
            'rre' : {},
            'rrh' : {},
            'rrhe' : {},
            'rt': {}, ##courtbouillonnai
            'gh' : {}, # ????
        },
        'fin' : {
            'r' : {},
            'rc' : {},
            'rcs' : {},
            'rd' : {},
            'rds' : {},
            're' : {},
            'rent' : {},
            'res' : {},
            'rf' : {},
            'rfs' : {},
            'rg' : {},
            'rgh' : {},
            'rgs' : {},
            'rps' : {},
            'rr' : {},
            'rre' : {},
            'rrent' : {},
            'rres' : {},
            'rrhe' : {},
            'rrhent' : {},
            'rrhes' : {},
            'rrs' : {},
            'rs' : {},
            'rt' : {},
            'rts' : {},
            'gh' : {},
            'ghe' : {},
            'ghes' : {},
            'ghs' : {},
        },
    },
    'l' : {
        'isolé' : {
            "l’" : {},
        },
        'début' : {
            'l' : {},
            'll' : {},
        },
        'milieu' : {
            'e' : {}, ##googleliser
            'gl' :{}, ##imbroglio
            'l' : {},
            'le' : {},
            'li' : {}, ##aliya
            'lh' : {},
            'll' : {},
            'lle' : {},
        },
        'fin' : {
            'e' : {}, ##battle
            'es' : {}, ##crumbles
            'l' : {},
            'le' : {},
            'lent' : {},
            'les' : {},
            'lle' : {},
            'llent' : {},
            'lles' : {},
            'll' : {},
            'lls' : {},
            'ls' : {},
        },
    },
    'j' : {
        'isolé' : {
            'y' : {},
            'y’' : {},
        },
        'début' : {
            'hi' : {},
            'hy' : {},
            'i' : {},
            'ï' : {},
            'j' : {},
            'y' : {},
            'yi' : {}, ##yiable
        },
        'milieu' : {
            'e' : {}, ##newton
            'g': {}, ## designer
            'i' : {},
            'ï' : {},
            'ïi' : {}, ##abaïiez
            'ill' : {},
            'ille' : {},
            'illi' : {}, ## désembouteillions
            'j' : {},
            'gh' : {}, ##copyright
            'gui' : {},
            'hi' : {}, ##bahianiser
            'l' : {},
            'lh' : {}, ##abeilhanais
            'll' : {},
            'lle' : {},
            'lli' : {}, ##abrilliez
            'n' : {}, ##bignoler
            'u' : {}, ##barbecue
            'y' : {},
            'ye' : {},
            'yi' : {},
        },
        'fin' : {
            'e' : {}, ##bye
            'glie' : {},
            'glies' : {},
            'i' : {},
            'ï' : {},
            'ie' : {}, ##barbéie
            'ïe' : {},
            'ïes' : {},
            'ïent' : {}, ##abaïent
            'il' : {},
            'ille' : {},
            'illes' : {},
            'ils' : {},
            'illent' : {},
            'is' : {},
            'ïs' : {},
            'lle' : {},
            'llent' : {},
            'lles' : {},
            'y' : {},
            'ye' : {},
            'yent' : {},
            'yes' : {},
            'ys' : {},
        },
    },
    'w' : {
        'isolé' : {
        },
        'début' : {
            'ho' : {},
            'hou' : {},
            'ou' : {},
            'w' : {},
            'o' : {}, ## oiseau
        },
        'milieu' : {
            'hou' : {},
            'hu' : {},
            'o' : {},
            'oo' : {},
            'ou' : {},
            'oy' : {}, ## broyement
            'u' : {},
            'w' : {},
        },
        'fin' : {
            'w' : {},
        },
    },
    'ɥ' : {
        'isolé' : {
        },
        'début' : {
            'hu' : {},
        },
        'milieu' : {
            'hu' : {},
            'u' : {},
            'ü' : {},
            '’hu' : {}, ##aujourd’hui
        },
        'fin' : {
        },
    },
    'i' : {
        'isolé' : {
            'hi': {},
            'hie': {},
            'hient': {},
            'hies': {},
            'y': {},
        },
        'début' : {
            'hi' : {},
            'hie' : {},
            'hy' : {},
            'i' : {},
            'î' : {},
            'y' : {},
        },
        'milieu' : {
            'e' : {},
            'ea' : {},
            'ee' : {},
            'eo' : {}, ## peopliser
            'hi' : {},
            'hî' : {},
            'hy' : {}, ##réhydrater
            'i' : {},
            'î' : {},
            'ï' : {},
            'ie' : {},
            'ie' : {},
            'is' : {}, ## disjoncté
            'o': {}, ## businesswomen
            'u' : {},
            'ui' : {}, ## bodybuiledrons
            'y' : {},
            'yg' : {}, ## amygdalienne
            'ÿ' : {},            
        },
        'fin' : {
            'ee' : {}, ## banoffee
            'ees': {}, ## burpees
            'hi' : {},
            'hie' : {},
            'hies' : {},
            'his' : {},
            'hit' : {},
            'hît' : {},
            'hy' : {},
            'i' : {},
            'ï' : {},
            'î' : {}, ## barakî
            'id' : {},
            'ids' : {},
            'ict' : {},
            'icts' : {},
            'ie' : {},
            'ïe' : {},
            'ient' : {},
            'ies' : {},
            'ïes' : {},
            #'iest' : {}, #Saint-Priest
            'ii' : {},
            'il' : {},
            'ils' : {},
            'is' : {},
            'ïs' : {},
            'it' : {},
            'ît' : {},
            'its' : {},
            'ix' : {},
            'iz' : {},
            'y' : {},
            'ÿ' : {},
            'ye' : {},
            'yes' : {},
            'yie' : {},
            'yies' : {},
            'ys' : {},
            'yt' : {},
        },
    },
    'e' : {
        'isolé' : {
            'ai' : {},
            'eh' : {},
            'et' : {},
            'hé' : {},
            'hés' : {},
        },
        'début' : {
            'ae': {}, ## aedicule
            'ai' : {}, ## aigri
            'aî' : {}, ## aîné
            'æ' : {},
            'ay' : {},
            'e' : {},
            'é' : {},
            'ey' : {},
            'ha' : {},
            'he' : {},
            'hé' : {},
            'œ' : {},
            'oe' : {}, ## oedipianiser
        },
        'milieu' : {
            'a' : {},
            'ae' : {}, ## alaeforme
            'æ' : {},
            'ai' : {},
            'ay' : {}, ## abraysien
            'e' : {},
            'é' : {},
            'ê' : {}, ## bêcher
            'ë' : {}, ## aërométrie
            'ée' : {},
            'ei' : {}, ## baleinier
            'ey' : {},
            'hé' : {},
            'œ' : {},
            'oe' : {}, ## réoedipianiser
        },
        'fin' : {
            'ae' : {},
            'æ' : {},
            'aes' : {},
            'ai' : {},
            'ais' : {}, ## bandolais
            'ay' : {}, ## gray
            'ays' : {}, ## yoctograys
            'e' : {}, ## aguardiente
            'é' : {},
            'ë' : {},
            'ed' : {},
            'eds' : {},
            'eh' : {},
            'ehe' : {},
            'ehes' : {},
            'ehs' : {},
            'ee' : {},
            'é' : {},
            'ë' : {},
            'ër' : {}, ## argüer
            'ed' : {},
            'eds' : {},
            'eh' : {},
            'ehe' : {},
            'ehes' : {},
            'ehs' : {},
            'ee' : {},
            'et' : {}, ## nordet
            'ets' : {}, ## bouchets (err Wix?)
            'ée' : {},
            'éent' : {},
            'ées' : {},
            'ef' : {},
            'efs' : {},
            'er' : {},
            'ers' : {},
            'es' : {},
            'és' : {},
            'ës' : {},
            'ey' : {}, ## baloney
            'ex' : {},
            'ez' : {},
            'her' : {}, ## bréher
        },
    },
    'ɛ' : {
        'isolé' : {
            'aie' : {},
            'aient' : {},
            'aies' : {},
            'ais' : {},
            'ait' : {},
            'ayent' : {},
            'es' : {},
            'est' : {},
            'haie' : {},
            'haies' : {},
            'hais' : {},
            'hait' : {},
            'haye' : {},
        },
        'début' : {
            'a' : {}, ## aces
            'ae' : {}, ## aerarium
            'æ' : {},
            'ai' : {},
            'aî' : {},
            'e' : {},       
            'é' : {},       
            'è' : {},
            'ê' : {},
            'ha' : {},
            'hay' : {},
            'he' : {},
            'hé' : {},
            'hê' : {},
            'œ' : {},
        },
        'milieu' : {
            'a' : {},
            'ä' : {}, ## bäckeofe
            'ae' : {},
            'æ' : {}, ## adhære
            'ai' : {},
            'aie' : {},
            'ay' : {},
            'aye' : {},
            'aî' : {},
            'e' : {},
            'ey' : {}, ## beylik
            'è' : {},
            'é' : {},
            'ê' : {},
            'ë' : {},
            'ea' : {},
            'ee' : {},
            'ei' : {},
            'es' : {},
            'hai' : {}, ## bréhaigne
            'he' : {},
            'hè' : {}, # bohème
            'i': {}, ## câliboire (Canada)
            'ue': {}, ## cuercir
            'œ' : {}, ## métœque
        },
        'fin' : {
            'ai' : {},
            'aict' : {},
            'aicts' : {},
            'aid' : {},
            'aids' : {},
            'aie' : {},
            'aient' : {},
            'aies' : {},
            'ais' : {},
            'ait' : {},
            'aît' : {},
            'aits' : {},
            'aix' : {},
            'ay' : {},
            'aye' : {},
            'ayes' : {},
            'ays' : {},
            'eah' : {}, ## yeah
            'ect' : {},
            'ects' : {},
            'eh' : {},
            'ehs' : {},
            'eigh' : {},
            'eighs' : {},
            'es' : {},
            'est' : {},
            'è' : {}, ## agapè
            'ès' : {},
            'et' : {},
            'ê' :  {}, ##corê
            'ês' :  {}, ##corês
            'êt' : {},
            'ets' : {},
            'êts' : {},
            'ey' : {},
            'eys' : {},
        },
    },
    'ɛ̃' : {
        'isolé' : {
            'aim' : {},
            'aims' : {},
            'ain' : {},
            'ains' : {},
            'ein' : {},
            'eins' : {},
            'haim' : {},
            'haims' : {},
            'hain' : {},
            'hains' : {},
            'hein' : {},
        },
        'début' : {
            'ain' : {},
            'ein' : {},
            'hin' : {},
            'im' : {},
            'in' : {},
        },
        'milieu' : {
            'ain' : {},
            'e' : {}, ## bienêtre
            'ein' : {},
            'en' : {}, ## bienvenue
            'im' : {},
            'in' : {},
            'ing' : {}, ## vingtuple
            'ïn': {}, ## coïnventiez
            'în' : {},
            'ym' : {},
            'yn' : {},
        },
        'fin' : {
            'aim' : {},
            'aims' : {},
            'ain' : {},
            'ainc' : {},
            'aincs' : {},
            'aing' : {},
            'aings' : {},
            'ains' : {},
            'aint' : {},
            'aints' : {},
            'ein' : {},
            'eins' : {},
            'eing' : {},
            'eings' : {},
            'eint' : {},
            'eints' : {},
            'en' : {},
            'ens' : {},
            'ent' : {},
            'in' : {},
            'inct' : {},
            'incts' : {},
            'ingt' : {},
            'ingts' : {},
            #'inq' : {}, #cinq-cent
            'ins' : {},
            'int' : {},
            'ints' : {}, ## adjoints
            'înt' : {},
            'ym' : {},
            'yms' : {},
            'yn' : {},
        },
    },    
    'œ̃' : {
        'isolé' : {
            'Hun' : {},
            'Huns' : {},
            'un' : {},
            'uns' : {},
        },
        'début' : {
            'hum' : {},
            'um' : {},
            #'un' : {}, #pas besoin dans unfollower 
        },
        'milieu' : {
            'un' : {},
        },
        'fin' : {
            'um' : {},
            'ums' : {},
            'un' : {},
            'uns' : {},
            'unt' : {},
            'unts' : {},
        },
    },
    'œ' : {
        'isolé' : {
        },
        'début' : {
            'heu' : {},
            'hœ' : {},
            'hu' : {},
            'œ' : {},
            'œu' : {},
            #'un' : {}, pas nécessaire dans unfollower
        },
        'milieu' : {
            'e' : {},
            'ea' : {}, ## yearling
            'eu' : {},
            'i' : {},
            'io' : {}, ## fashion
            'l' : {}, ## battle
            'œ' : {},
            'heu' : {}, ## aheurter
            'o' : {}, ## patchworkiser
            'œu' : {},
            'ogl' : {},
            'oe' : {}, ## arroeiller
            'u' : {},
            'ue' : {},
        },
        'fin' : {
            'eu' : {},
            'euent' : {},
            #'eues' : {},
        },
    },
    'ə' : {
        'isolé' : {
        },
        'début' : {
        },
        'milieu' : {
            'ai' : {},
            'e' : {},
            'on' : {},
        },
        'fin' : {
            'e' : {},
        },
    },    
    'ø' : {
        'isolé' : {
            'euh' : {},
            'eux' : {},
            'œufs' : {},
        },
        'début' : {
            'e' : {}, #EDF
            'eu' : {},
            'heu' : {},
            'hœ' : {},
            'œ' : {},
        },
        'milieu' : {
            'e': {}, ## grelu
            'eu' : {},
            'eû' : {},
            'oe' : {},
            'œ' : {},
            'œu' : {}, ## bœufs
            'ö' : {}, ## börek
        },
        'fin' : {
            'eu' : {},
            'eud' : {}, ## alleud
            'eue' : {},
            'euent' : {},
            'eues' : {},
            'euf' : {}, ## éteuf
            'eufs' : {}, ## éteufs
            'euh' : {},
            'euhe' : {},
            'euhes' : {},
            'euhs' : {},
            'eur' : {},
            'eurs' : {},
            'eus' : {},
            'eut' : {},
            'euts' : {},
            'eux' : {},
            'œu' : {},
            'œud' : {},
            'œuds' : {},
            'œulx' : {},
            'œufs' : {}, ## bœufs
            'œux' : {},
        },
    },    
    'y' : {
        'isolé' : {
            'eu' : {},
            'eue' : {},
            'eues' : {},
            'eus' : {},
            'eut' : {},
            'eût' : {},
            'hue' : {},
            'huent' : {},
            'hues' : {},
        },
        'début' : {
            'eu' : {},
            'eû' : {},
            'hu' : {},
            'u' : {},
            'ü' : {},
        },
        'milieu' : {
            'eu' : {},
            'eü' : {},
            'hu' : {},
            'u' : {},
            'us' : {}, ## brusler
            'uu' : {}, ## buuz
            'ü' : {},
            'üe' : {}, ## argüerai
            'uë' : {}, ## arguëra
            'û' : {},
            'ue' : {},
        },
        'fin' : {
            'hu' : {},
            'hue' : {},
            'huent' : {},
            'hues' : {},
            'hus' : {},
            'hut' : {},
            'huts' : {},
            'u' : {},
            'ü' : {},
            'û' : {},
            'ue' : {},
            'uë' : {},
            'üe' : {},
            'uent' : {},
            'uënt' : {}, ## arguënt
            'üent' : {}, ## argüent
            'ues' : {},
            'uës' : {},
            'ües' : {},
            'ul' : {},
            'uls' : {},
            'upt' : {},
            'upts' : {},
            'us' : {},
            'ûs' : {},
            'ut' : {},
            'ût' : {},
            'uth' : {},
            'uths' : {},
            'uts' : {},
            'ûts' : {},
            'ux' : {},
            'uz' : {},
        },
    },    
    'u' : {
        'isolé' : {
            'hou' : {},
            'houe' : {},
            'houx' : {},
            'ou' : {},
            'où' : {},
            'ouh' : {},
        },
        'début' : {
            'aou' : {},
            'aoû' : {},
            'hou' : {},
            'hu' : {},
            'ou' : {},
            'oû' : {},
            'u' : {},
            'ü' : {},
        },
        'milieu' : {
            'aou' : {},
            'e' : {}, ## autointerviewa
            'hou' : {}, ## déhoussage
            'o' : {},
            'oe' : {}, ## broebèler (belgique)
            'oo' : {},
            'ou' : {},
            'oup' : {}, ## coupdebouliser
            'oû' : {},
            'oue' : {},
            'ow' : {},
            'u' : {},
            'ui' : {}, ## cruiser
            'ü' : {},
            'w' : {}, ## newton
        },
        'fin' : {
            'aoul' : {},
            'aouls' : {},
            'e': {}, ## barbecue
            'ew' : {},
            'ews' : {},
            'ou' : {},
            'où' : {}, ## ankoù
            'oub' : {},
            'oubs' : {},
            'ouc' : {},
            'oucs' : {},
            'oud' : {},
            'ouds' : {},
            'oue' : {},
            'ouent' : {},
            'ouh' : {},
            'oues' : {},
            'oug' : {},
            'ougs' : {},
            'ouls' : {}, ## pouls
            'oo' : {},
            'oos' : {},
            'oup' : {},
            'oups' : {},
            'ous' : {},
            'out' : {},
            'oût' : {},
            'outs' : {}, ## abouts
            'oûts' : {},
            'oux' : {},
            'u' : {},
            'us' : {},
        },
    },
    'o' : {
        'isolé' : {
            'au' : {},
            'aulx' : {},
            'aux' : {},
            'eau' : {},
            'eaux' : {},
            'haut' : {},
            'hauts' : {},
            'ho' : {},
            'ô' : {},
            'oh' : {},
            'os' : {},
        },
        'début' : {
            'au' : {},
            'aul' : {}, ## aulne
            'aux' : {}, ## auxdits
            'eau' : {},
            'eaux' : {},
            'ha' : {}, ## hall
            'hau' : {},
            'haut' : {},
            'hauts' : {},
            'heau' : {},
            'ho' : {},
            'hô' : {},
            'hoa' : {},
            'o' : {},
            'ô' : {},
            'oh' : {},
        },
        'milieu' : {
            'a' : {},
            'au' : {},
            'aû': {}, ## afaûtri
            'aw' : {}, ## crawlera
            'eo' : {}, ## yeoman
            'eau' : {}, ## beauvotte
            'hau' : {},
            'o' : {},
            'ô' : {},
            'ó' : {}, ## yóm
            'ō' : {}, ## yōkai
            'oa' : {},
            'oh' : {}, ## ohm
            'oo' : {},
        },
        'fin' : {
            'ao' : {},
            'aos' : {},
            'au' : {},
            'aud' : {},
            'auds' : {},
            'ault' : {},
            'aults' : {},
            'aulx' : {},
            'aus' : {},
            'aut' : {},
            'auts' : {},
            'auw' : {},
            'auws' : {},
            'aux' : {},
            'aw' : {},
            'aws' : {},
            'eau' : {},
            'eaus' : {},
            'eaux' : {},
            'hau' : {},
            'haut' : {},
            'hauts' : {},
            'ho' : {},
            'hos' : {},
            'o' : {},
            'ô' : {},
            'oc' : {},
            'ocs' : {},
            'oe' : {},
            'oh' : {},
            'oo' : {},
            'oos' : {},
            'ooz' : {},
            'op' : {},
            'ops' : {},
            'os' : {},
            'ôs' : {},
            'ot' : {},
            'ôt' : {},
            'oth' : {},
            'oths' : {},
            'ots' : {},
            'ôts' : {},
            'ow' : {},
            'ows' : {},
            'oz' : {},
        },
    },
    'ɔ' : {
        'isolé' : {
        },
        'début' : {
            'au' : {}, ## autocondionné
            'ho' : {},
            'o' : {},
        },
        'milieu' : {
            'a' : {},
            'ao' : {},
            'au' : {},
            'aw' : {}, ## yawl
            'ho' : {}, ## réhomologuerions
            'o' : {},
            'ö' : {}, ## coöpèrent
            'ô' : {}, ## yôghi
            'ō': {}, ## yōon
            'oa' : {},
            'oho' : {}, ## alcohol
            'oo' : {},
            'ou' : {},
            'ow' : {}, ## crowdsourcé
            'u' : {},
            'ü' : {},
            'œ' : {}, ## cœrcif
        },
        'fin' : {
            'o' : {}, ## agitato ###should not happen, err in Wix
            'os' : {}, ## allégrettos
            'ot' : {}, ## burot (err Wix?)
            'ots' : {}, ## burots (err Wix?)
        },
    },
    'ɔ̃' : {
        'isolé' : {
            'hon' : {},
            'on' : {},
            'ont' : {},
        },
        'début' : {
            'hon' : {},
            'hom' : {},
            'om' : {},
            'on' : {},
            'um' : {},
            'un' : {},
        },
        'milieu' : {
            'hon' : {},
            'om' : {},
            'omp' : {},
            'on' : {},
            'um' : {},
            'ump' : {},
            'un' : {},
            'unk' : {},
        },
        'fin' : {
            'aon' : {},
            'aons' : {},
            'om' : {},
            'omb' : {},
            'ombs' : {},
            'ompt' : {},
            'ompts' : {},
            'oms' : {},
            'on' : {},
            'onc' : {},
            'oncs' : {},
            'ond' : {},
            'onds' : {},
            'ong' : {},
            'ongs' : {},
            'ons' : {},
            'ons' : {},
            'ont' : {},
            'onts' : {},
        },
    },
    'ɑ̃' : {
        'isolé' : {
            'an' : {},
            'ans' : {},
            'en' : {},
            'han' : {},
        },
        'début' : {
            'e' : {}, # XM for enharmonie
            'am' : {},
            'an' : {},
            'em' : {},
            'en' : {},
            'ham' : {},
            'han' : {},
            'hen' : {},
        },
        'milieu' : {
            'a' : {}, ##bamba
            'aan' : {},
            'aën' : {},
            'am' : {},
            'amp' : {}, ##champlever
            'an' : {},
            'ang' : {}, ##bangkokien
            'e' : {}, ##renamourer
            'em' : {},
            'emp' : {},
            'en' : {},
            'han' : {},
            'hen' : {},
        },
        'fin' : {
            'aen' : {},
            'am' : {},
            'amp' : {},
            'amps' : {},
            'ams' : {},
            'an' : {},
            'anc' : {},
            'ancs' : {},
            'and' : {},
            'ands' : {},
            'ang' : {},
            'angs' : {},
            'ans' : {},
            'ant' : {},
            'ants' : {},
            'aon' : {},
            'aons' : {},
            'emps' : {},
            'empt' : {},
            'empts' : {},
            'end' : {},
            'ends' : {},
            'eng' : {},
            'engs' : {},
            'ens' : {},
            'ent' : {},
            'ents' : {},
            'han' : {},
            'hans' : {},
        },
    },
    'ɑ' : {
        'isolé' : {
        },
        'début' : {
            'a': {}, ##affrontation (probably an error in Wiktionary)
            'â' : {},
            'hâ' : {},
        },
        'milieu' : {
            'a' : {}, ##enfutailler
            'as' : {}, ##basfonds
            'â' : {},
            'aa' : {},
            'am' : {},
            'âh' : {}, ##brâhmane
            'i' : {}, ##reboiserait
            'î' : {}, ##boîter
        },
        'fin' : {
            'as' : {},
            'âs' : {},
            'ât' : {},
            'at' : {}, ##agnaca (Wix err)
            'ats' : {}, ##agnacas (Wix err)
            'âts' : {},
            'aw' : {}, ##bucksaw
            'az' : {},
        },
    },
    'a' : {
        'isolé' : {
            'a' : {},
            'à' : {},
            'ah' : {},
            'as' : {},
        },
        'début' : {
            'a' : {},
            'aa' : {},
            'ah' : {}, ##ahrimane
            'as' : {}, ##asnierois
            'ha' : {},
            'hâ' : {}, ##hâler
        },
        'milieu' : {
            'a' : {},
            'â' : {},
            'ã' : {}, ##baião
            'aa' : {},
            'ae' : {},
            'ah' : {}, ##bahreïni
            'ao' : {},
            'as' : {}, ##basbleuisa
            'at' : {}, ##étatsunianiser
            'e' : {},
            'ê' : {},
            'ha' : {},
            'hâ' : {}, ##déhâler
            'i' : {},
            'î' : {},
            'ie' : {},
            'ig' : {},
            'o' : {}, ##burnouter
            'y' : {}, ##bye
        },
        'fin' : {
            'a' : {},
            'à' : {},
            'āʾ' : {}, ##yāʾ
            'aa' : {},
            'ac' : {},
            'ach' : {},
            'achs' : {},
            'acs' : {},
            'act' : {},
            'acts' : {},
            'ah' : {},
            'ahs' : {},
            'ap' : {},
            'aps' : {},
            'ars' : {},
            'as' : {},
            'at' : {},
            'ât' : {},
            'ats' : {},
            'âts' : {},
            'ax' : {},
            'ha': {}, ##baha
            'i' : {},
            'id' : {},
            'ids' : {},
            'ie' : {},
            'ient' : {},
            'ies' : {},
            'igt' : {},
            'igts' : {},
            'is' : {},
            'it' : {},
            'ît' : {},
            'its' : {},
            'îts' : {},
            'ix' : {},
            'ois' : {}, ##aderguois 
            'yes' : {},
        },
    },
    ## gngn de phonemes multiples pointant sur 1 seul et même graphème
    'aj' : {
        'isolé' : {
        },
        'début' : {
            'hei' : {}, ##heideggérien
        },
        'milieu' : {
            'hei' : {}, ##désheideggérianiser
            'hi' : {}, ##déwhitelister
            'i' : {}, ##striker
            'y' : {}, ##flysurf
        },
        'fin' : {            
        },
    },   
    'dʒ' : {
        'isolé' : {
        },
        'début' : {
            'j' : {}, ##jaina
        },
        'milieu' : {
            'g' : {}, ##challenger
        },
        'fin' : {            
        },
    },       
    'ij' : {
        'isolé' : {
        },
        'début' : {
        },
        'milieu' : {
            'i' : {}, ##cypriote
        },
        'fin' : {            
        },
    },   
    'ks' : {
        'isolé' : {
        },
        'début' : {
            'x' : {}, ##xanthopie
        },
        'milieu' : {
            'x' : {}, ##excellent
            'xc' : {}, ##excité
        },
        'fin' : {
            'x': {}, ##solex
            'xe' : {}, ##luxe
            'xes' : {}, ##luxes
            'xent' : {}, ##fixent
        },
    },   
    'ɡz' : {
        'isolé' : {
        },
        'début' : {
            'x' : {} ##xénophobe
        },
        'milieu' : {
            'x' : {}, ##excellent
        },
        'fin' : {
        },
    },  
    'ɔɛ' : {
        'isolé' : {
        },
        'début' : {
        },
        'milieu' : {
            'œ' : {}, ##cœrcif
        },
        'fin' : {
        },
    },  
    'tʃ' : {
        'isolé' : {
        },
        'début' : {
            'ch' : {}, ##chatter
            'x' : {}, ##xavante
        },
        'milieu' : {
            'ch' : {}, ##rechatter
            'œ' : {}, ##acehnais
        },
        'fin' : {
        },
    },  
    'wa' : {
        'isolé' : {
        },
        'début' : {
        },
        'milieu' : {
            'o' : {}, ##ondoyés
        },
        'fin' : {
        },
    },   
    ## phonemes utilisés pour prononcer l'alphabet
    ## i.e. phonèmes multiples pointant sur un seul et même graphème
    'be' : {
        'isolé' : {
            'b': {}, 
        },
        'début' : {
            'b': {},
        },
        'milieu' : {
            'b' : {},
        },
        'fin' : {
            'b' : {},
        },
    }, 
    'se' : { 
        'isolé' : {
            'c': {}, 
        },
        'début' : {
            'c': {},
        },
        'milieu' : {
            'c' : {},
        },
        'fin' : {
            'c' : {},
        },
    }, 
    'de' : {
        'isolé' : {
            'd': {}, 
        },
        'début' : {
            'd': {},
        },
        'milieu' : {
            'd' : {},
        },
        'fin' : {
            'd' : {},
        },
    }, 
    'ɛf' : {
        'isolé' : {
            'f': {}, 
        },
        'début' : {
            'f': {},
        },
        'milieu' : {
            'f' : {},
        },
        'fin' : {
            'f' : {},
        },
    }, 
    'ʒe' : {
        'isolé' : {
            'g': {}, 
        },
        'début' : {
            'g': {},
        },
        'milieu' : {
            'g' : {},
        },
        'fin' : {
            'g' : {},
        },
    }, 
    'aʃ' : {
        'isolé' : {
            'h': {}, 
        },
        'début' : {
            'h': {},
        },
        'milieu' : {
            'h': {},
        },
        'fin' : {
            'h': {},
        },
    }, 
    'ʒi' : {
        'isolé' : {
            'j': {}, 
        },
        'début' : {
            'j': {},
        },
        'milieu' : {
            'j' : {},
        },
        'fin' : {
            'j' : {},
        },
    }, 
    'ka' : {
        'isolé' : {
            'k': {}, 
        },
        'début' : {
            'k': {},
        },
        'milieu' : {
            'k' : {},
        },
        'fin' : {
            'k' : {},
        },
    }, 
    'ɛl' : {
        'isolé' : {
            'l': {},
        },
        'début' : {
            'l': {}, 
        },
        'milieu' : {
            'l': {},
        },
        'fin' : {
            'l': {},
        },
    }, 
    'ɛm' : {
        'isolé' : {
            'm': {}, 
        },
        'début' : {
            'm': {},
        },
        'milieu' : {
            'm': {},
        },
        'fin' : {
            'm': {},
        },
    }, 
    'ɛn' : {
        'isolé' : {
            'n': {}, 
        },
        'début' : {
            'n': {},
        },
        'milieu' : {
            'n': {},
        },
        'fin' : {
            'n': {},
        },
    }, 
    'pe' : {
        'isolé' : {
            'p': {}, 
        },
        'début' : {
            'p': {},
        },
        'milieu' : {
            'p': {},
        },
        'fin' : {
            'p': {},
        },
    }, 
    'ky' : {
        'isolé' : {
            'q': {}, 
        },
        'début' : {
            'q': {},
        },
        'milieu' : {
            'q': {},
        },
        'fin' : {
            'q': {},
        },
    }, 
    'ɛʁ' : {
        'isolé' : {
            'r': {}, 
        },
        'début' : {
            'r': {},
        },
        'milieu' : {
            'r': {},
        },
        'fin' : {
            'r': {},
        },
    }, 
    'ɛs' : {
        'isolé' : {
            's': {}, 
        },
        'début' : {
            's': {},
        },
        'milieu' : {
            's': {},
            'ss': {}, # SMSsera
        },
        'fin' : {
            's': {},
        },
    }, 
    'te' : {
        'isolé' : {
            't': {},
        },
        'début' : {
            't': {}, 
        },
        'milieu' : {
            't': {},
        },
        'fin' : {
            't': {},
        },
    }, 
    've' : {
        'isolé' : {
            'v': {},
        },
        'début' : {
            'v': {}, 
        },
        'milieu' : {
            'v': {},
        },
        'fin' : {
            'v': {},
        },
    }, 
    'dubləve' : {
        'isolé' : {
            'w': {},
        },
        'début' : {
            'w': {}, 
        },
        'milieu' : {
            'w': {},
        },
        'fin' : {
            'w': {},
        },
    }, 
    'iks' : {
        'isolé' : {
            'x': {}, 
        },
        'début' : {
            'x': {}, 
        },
        'milieu' : {
            'x': {},
        },
        'fin' : {
            'x': {},
        },
    },    
    'iɡʁɛk' : {
        'isolé' : {
            'y': {}, 
        },
        'début' : {
            'y': {}, 
        },
        'milieu' : {
            'y': {},
        },
        'fin' : {
            'y': {},
        },
    }, 
    'zɛd' : {
        'isolé' : {
            'z': {},
        },
        'début' : {
            'z': {}, 
        },
        'milieu' : {
            'z': {},
        },
        'fin' : {
            'z': {},
        },
    },   
}

# creation 'un fonction de correspondance ne prenant pas en compte la position
# du phoneme dans le mot. Une telle table permet d'analyser plus facilement les
# mots composés (ie. avec un espace ou un tiret) étant donné que la fin d'un 
# mot simple peut se retrouver au milieu d'un mot composé
def create_phonemes2graphemes(phonemes2graphemes_old):

  phonemes2graphemes = {}

  '''Ajoute des clés ('occurences', 'exemple_1', 'exemple_2', 'exemple_3')
   dans le dictionnaire de chaque grapheme de la table phonemes2graphemes.'''
  for phoneme in phonemes2graphemes_old.keys():
    phonemes2graphemes[phoneme] = {}

    for position in phonemes2graphemes_old[phoneme].keys():

      for grapheme in phonemes2graphemes_old[phoneme][position].keys():
        phonemes2graphemes[phoneme][grapheme] = {}
        phonemes2graphemes[phoneme][grapheme]['occurences'] = 0
        phonemes2graphemes[phoneme][grapheme]['exemple_1'] = ''
        phonemes2graphemes[phoneme][grapheme]['exemple_2'] = ''
        phonemes2graphemes[phoneme][grapheme]['exemple_3'] = ''

  return phonemes2graphemes

phonemes2graphemes = create_phonemes2graphemes(phonemes2graphemes)

## Fonctions de comptage pour statistiques

In [9]:
# fonction de comptage qui incrémente d'une unité le compteur de correspondance
# entre un phonème donné et un graphème donné et qui enregistre en exemple 
# les trois premiers mots dans lequels la correspondance a été trouvé 
def increment_occurences(phoneme, grapheme, mot, unit_test=False):
  
  global phonemes2graphemes

  # ne pas continuer si l'appel provient d'un test unitaire
  if unit_test:
    return

  phonemes2graphemes[phoneme][grapheme]['occurences']  += 1
  if phonemes2graphemes[phoneme][grapheme]['exemple_1']  == '':
    phonemes2graphemes[phoneme][grapheme]['exemple_1'] = mot
  elif phonemes2graphemes[phoneme][grapheme]['exemple_2']  == '':
    phonemes2graphemes[phoneme][grapheme]['exemple_2'] = mot
  elif phonemes2graphemes[phoneme][grapheme]['exemple_3']  == '':
    phonemes2graphemes[phoneme][grapheme]['exemple_3'] = mot

In [10]:
# fonction de statistiques qui calcule pour pour chaque position (initiale,
# intermédiaire ou finale) le nombre d'occurences de chaque graphème ainsi
# que la probabilité de ce graphème.
# Les résultats sont insérés dans un DataFrame pandas et dans un fichier CSV.

def make_stats():
  global phonemes2graphemes

  table = []
  for phoneme in phonemes2graphemes:
      for grapheme in phonemes2graphemes[phoneme].keys():
        occurences = phonemes2graphemes[phoneme][grapheme]['occurences']
        exemple_1 = phonemes2graphemes[phoneme][grapheme]['exemple_1']
        exemple_2 = phonemes2graphemes[phoneme][grapheme]['exemple_2']
        exemple_3 = phonemes2graphemes[phoneme][grapheme]['exemple_3']

        #print("phoneme=%s grapheme=%s occurences=%d" % (phoneme, grapheme, occurences))
        row = { 'phonème': phoneme, 'graphème':grapheme, 'occurences':occurences, 'pourcentage':0.0,
               'exemple_1' : exemple_1, 'exemple_2' : exemple_2, 'exemple_3' : exemple_3, }
        table.append(row)
  dfp = pd.DataFrame.from_dict(table)

  for phoneme in dfp['phonème'].unique():
      df1 = dfp[dfp['phonème'] == phoneme]

      # calculate the sum of used graphemes within each phoneme-position category
      sum = 0
      for grapheme in df1['graphème'].unique():
        row_number = df1.loc[df1.graphème==grapheme].index[0]
        occurences = df1.at[row_number, 'occurences']
        sum += occurences

      # set the pourcentage of each graphemes for each phoneme-position category
      if sum != 0:
        for grapheme in df1['graphème'].unique():
          row_number = df1.loc[df1.graphème==grapheme].index[0]
          occurences = df1.at[row_number, 'occurences']
          pourcentage = int(occurences/sum*100)/100
          dfp.at[row_number, 'pourcentage'] = pourcentage

      dfp.to_csv("phonemes2graphemes.csv", index=False)

  return dfp

make_stats()

,phonème,graphème,occurences,pourcentage,exemple_1,exemple_2,exemple_3
0,m,m’,0,0.0,,,
1,m,hm,0,0.0,,,
2,m,m,0,0.0,,,
3,m,me,0,0.0,,,
4,m,mm,0,0.0,,,
...,...,...,...,...,...,...,...
962,ve,v,0,0.0,,,
963,dubləve,w,0,0.0,,,
964,iks,x,0,0.0,,,
965,iɡʁɛk,y,0,0.0,,,


## Fonction de test de plausibilité

In [11]:
# Fonction récursive tentant de retrouver tout ou partie des graphemes
# depuis tout ou partie des phonemes, à l'aide de la table de correspondance
# entre les phonemes et les graphemes.

def process_graphemes(mot, phonemes, graphemes, 
                      phoneme_index=0, grapheme_index=0, 
                      phonemes_hist='', graphemes_hist='', 
                      unit_test = False, verbose=False,):
  
  if verbose:
    print('')
    print('process_graphemes(phonemes=\\%s\\, graphemes=[[%s]], \
    phoneme_index=%d, grapheme_index=%d, \
    phonemes_hist=%s, graphemes_hist=%s)' 
            % (phonemes, graphemes, phoneme_index, grapheme_index, phonemes_hist, graphemes_hist))

  global phonemes2graphemes

  if grapheme_index >= len(graphemes):
    if ''.join(graphemes_hist) == graphemes and ''.join(phonemes_hist)==phonemes:
        if verbose:
            print('ended ok')            
        return True, phonemes_hist.copy(), graphemes_hist.copy()
    else:
        if verbose:
            print('ended ko')
        return False, phonemes_hist, graphemes_hist

  current_candidate_phonemes = []
  for phoneme in phonemes2graphemes:    
    if phonemes[phoneme_index:].startswith(phoneme):      
      current_candidate_phonemes.append(phoneme)
  if len(current_candidate_phonemes) == 0:
    return False, phonemes_hist, graphemes_hist
  if verbose:
    if len(current_candidate_phonemes) > 1:
      print("### candidate current phonemes=%", current_candidate_phonemes)

  nb_current_candidate_phonemes = 0 

  for current_phoneme in current_candidate_phonemes:
    if verbose:
      print("trying candidate current phoneme=%s" % current_phoneme)
    nb_current_candidate_phonemes += 1
    next_phoneme = ''

    last_graphemes = graphemes[grapheme_index:]
    
    if verbose:
      print('process_graphemes(phonemes=\\%s\\, graphemes=[[%s]], phoneme_index=%d, grapheme_index=%d) current_phoneme:%s' 
            % (phonemes, graphemes, phoneme_index, grapheme_index, current_phoneme))
  
    if verbose:
      print('current_phoneme:%s' % current_phoneme)
    
    try:
      len(phonemes2graphemes[current_phoneme])
    except:
      a=1
      if current_phoneme == '':
        if verbose:
          print('wrn: in [[%s]] : phoneme \\%s\\ does not exist !!!' % (graphemes, current_phoneme))
        return True, phonemes_hist, graphemes_hist
      else:
        print('err: in [[%s]] : phoneme \\%s\\ does not exist !!!' % (graphemes, current_phoneme))
        return False, phonemes_hist, graphemes_hist

    matching_graphemes_list = []

    for current_graphemes in phonemes2graphemes[current_phoneme]:
    
      if last_graphemes.startswith(current_graphemes):
        if verbose:
          print('phoneme \\%s\\ matching graphemes [[%s]] (last_graphemes:[[%s]])' % (current_phoneme, current_graphemes, last_graphemes))
        matching_graphemes_list.append(current_graphemes)

    if len(matching_graphemes_list) == 0:
      if verbose:
        print('KO0: phonemes=\\%s\\ does not match any grapheme !!!' % (phonemes))

      if nb_current_candidate_phonemes == len(current_candidate_phonemes):
        return False, phonemes_hist, graphemes_hist
      else:
        if verbose:
          print('move0 to next candidate phoneme')
        continue
      
    #print('current_phoneme=', current_phoneme)
    
    ### * ###
    if len(matching_graphemes_list) == 0:
        if verbose:
          print('KO: phoneme \\%s\\ matches NO grapheme (in word [[%s]])  !!!' % (graphemes, phonemes))
        phonemes_hist.pop()
        graphemes_hist.pop()
        return False, phonemes_hist, graphemes_hist

    for current_graphemes in matching_graphemes_list:
        
        phonemes_hist.append(current_phoneme)
        graphemes_hist.append(current_graphemes)

        if verbose:
          print("trying2 current_phoneme \\%s\\ and current_graphemes [[%s]]" % (current_phoneme, current_graphemes))

        ret, phonemes_hist2, graphemes_hist2 = process_graphemes(
            mot,
            phonemes, 
            graphemes,
            phoneme_index=phoneme_index+len(current_phoneme), 
            grapheme_index=grapheme_index+len(current_graphemes),
            phonemes_hist = phonemes_hist,
            graphemes_hist = graphemes_hist,
            unit_test = unit_test,
            verbose=verbose)           
                
        # si la concatenation des graphemes trouvées jusqu'ici n'est pas bonne, alors pas bon
        if not graphemes.startswith(''.join(graphemes_hist)):
            if verbose:
                print(''.join(graphemes_hist)+current_graphemes)
                print(graphemes)
                print('current_graphemes does not match previous findings')
            ret = False          
                
          # si la concatenation des graphemes trouvées jusqu'ici n'est pas bonne, alors pas bon

      
        if ret == True:
          if verbose:
            print('phonemes_hist2b:', phonemes_hist2)
            print('graphemes_hist2b:', graphemes_hist2)
            print("current_phoneme \\%s\\ and current_graphemes [[%s]] : worked ok" % (current_phoneme, current_graphemes))
          
          increment_occurences(current_phoneme, current_graphemes, mot, unit_test)
          return True, phonemes_hist2, graphemes_hist2
    
        phonemes_hist.pop()
        graphemes_hist.pop()
      
    # if here LOST
    if verbose:
          print('LOST2 for current_phoneme \\%s\\ in process_graphemes(phonemes=\\%s\\, graphemes=[[%s]], phoneme_index=%d, grapheme_index=%d)' 
                % (current_phoneme, phonemes, graphemes, phoneme_index, grapheme_index))
    
    if nb_current_candidate_phonemes == len(current_candidate_phonemes):
        return False, phonemes_hist2, graphemes_hist2
    else:
       
        if verbose:
          print('move2 to next candidate phoneme')
        continue

      

### Tests

### Exemples

In [12]:
exemples = [
    ("momie", "mɔmi"),
    ("pookie", "puki"),

]

for exemple in exemples:
    
    mot = exemple[0]
    graphemes = exemple[0]
    phonemes = exemple[1]
    
    res = process_graphemes(mot, phonemes, graphemes, 
                            phoneme_index=0, grapheme_index=0, 
                            phonemes_hist=[], graphemes_hist=[],
                            unit_test=True, verbose=False)

In [13]:
def check_word_pronunciation(mot, phonemes, graphemes, unit_test=False, verbose0=False, verbose1=False, verbose2=False):

    res, phonemes2, graphemes2  = process_graphemes(mot, phonemes, graphemes, phoneme_index=0, grapheme_index=0, phonemes_hist=[], graphemes_hist=[], unit_test=unit_test, verbose=verbose2)
    prononciation = ''.join(phonemes2)
    word = ''.join(graphemes2)
    prononciation_ = ','.join(phonemes2)
    word_ = ','.join(graphemes2)
    if prononciation != phonemes:
      if verbose0:
        print('[[%s]] \\%s\\ -> prononciation=\\%s\\ ' % (mot, phonemes, prononciation ))
      return False
    elif word != graphemes.replace('-','').replace(' ',''):
      if verbose0:
        print('[[%s]] != output graphemes=%s' % (mot, word))
      return True
    else:
      if verbose0:
        print('%s \\%s\\ -> %s \\%s\\' % (mot, prononciation, word_, prononciation_))
      return True

### Tests de référence

In [14]:
essais = [
    ("acéphalobrache", "asefalɔbʁak"),
    ("acquaintance","akwɛ̃tɑ̃s"),
    ("agoua","aɡwa"),
    ("bêchâmes","beʃam"),
    ("caouanne","kawan"),
    ("designer","dizajnœʁ"),
    ("désembouteillions","dezɑ̃butɛjɔ̃"),
    ("disjonctés","diʒɔ̃kte"),
    ("droitismes", "dʁwatism"),
    ("élégante", "eleɡɑ̃t"),
    ("enarrhas", "ɑ̃naʁa"),
    ("enfutailler", "ɑ̃fytɑje"),
    ("excaverai", "ɛkskavəʁe"),
    ("exemple", "ɛɡzɑ̃pl"),
    ("fashion","faʃœn"),
    ("khartoumisé","kaʁtumize"),
    ("hauts", "o"),
    ('hauts-fonds','o.fɔ̃'),
    ("html", "aʃteɛmɛl"), #acronyme
    ("intérêt", "ɛ̃teʁɛ"),
    ("intéressant","ɛ̃teʁɛsɑ̃"),
    ("jaïnas","dʒaina"),
    ("luxe","lyks"),
    ("momie","mɔmi"),
    ("oiseaux", "wazo"),
    ("ondoyés", "ɔ̃dwaje"),
    ("peopliser","piplize"),
    ("pookie", "puki"),
    ("rechaterais", "ʁətʃatəʁɛ"),
    ('solex', 'solɛks'),
    ('Paris', 'pa.ʁi'),
    ('VPN', 've.pe.ɛn'),
    
]

for essai in essais:
  #check_word_pronunciation(essai[0], essai[1].replace('.',''), essai[0].lower().replace('-',''), unit_test=True, verbose0=True, verbose1=False, verbose2=False)
  check_word_pronunciation(essai[0], essai[1].replace('.',''), essai[0].lower().replace('-',''), unit_test=True, verbose0=True, verbose1=False, verbose2=False)

acéphalobrache \asefalɔbʁak\ -> a,c,é,ph,a,l,o,b,r,a,che \a,s,e,f,a,l,ɔ,b,ʁ,a,k\
acquaintance \akwɛ̃tɑ̃s\ -> a,cq,u,ain,t,an,ce \a,k,w,ɛ̃,t,ɑ̃,s\
agoua \aɡwa\ -> a,g,ou,a \a,ɡ,w,a\
bêchâmes \beʃam\ -> b,ê,ch,â,mes \b,e,ʃ,a,m\
caouanne \kawan\ -> c,a,ou,a,nne \k,a,w,a,n\
designer \dizajnœʁ\ -> d,e,s,i,g,n,e,r \d,i,z,a,j,n,œ,ʁ\
désembouteillions \dezɑ̃butɛjɔ̃\ -> d,é,s,em,b,ou,t,e,illi,ons \d,e,z,ɑ̃,b,u,t,ɛ,j,ɔ̃\
disjonctés \diʒɔ̃kte\ -> d,is,j,on,c,t,és \d,i,ʒ,ɔ̃,k,t,e\
droitismes \dʁwatism\ -> d,r,o,i,t,i,s,mes \d,ʁ,w,a,t,i,s,m\
élégante \eleɡɑ̃t\ -> é,l,é,g,an,te \e,l,e,ɡ,ɑ̃,t\
enarrhas \ɑ̃naʁa\ -> e,n,a,rrh,as \ɑ̃,n,a,ʁ,a\
enfutailler \ɑ̃fytɑje\ -> en,f,u,t,a,ill,er \ɑ̃,f,y,t,ɑ,j,e\
excaverai \ɛkskavəʁe\ -> e,x,c,a,v,e,r,ai \ɛ,ks,k,a,v,ə,ʁ,e\
exemple \ɛɡzɑ̃pl\ -> e,x,em,p,le \ɛ,ɡz,ɑ̃,p,l\
fashion \faʃœn\ -> f,a,sh,io,n \f,a,ʃ,œ,n\
khartoumisé \kaʁtumize\ -> k,ha,r,t,ou,m,i,s,é \k,a,ʁ,t,u,m,i,z,e\
hauts \o\ -> hauts \o\
hauts-fonds \ofɔ̃\ -> hauts,f,onds \o,f,ɔ̃\
html \aʃteɛmɛl\ -> h,

### Test en masse

In [15]:
df.shape

(1484043, 8)

In [16]:
# trier le dataframe (noms communs, acronymes, noms propres, puis restants)
# pour des exemples plus significatifs dans les statistiques
df_composés = df[df.Mot.str.match('..*?[\-\ ].*?.')].copy()
print("nb mots composés:%d" % df_composés.shape[0])

df_autres_0 = pd.concat([df, df_composés, df_composés]).drop_duplicates(keep=False)
df_préfixes = df_autres_0[df_autres_0.Mot.str.startswith('-')].copy()
print("nb mots préfixes:%d" % df_préfixes.shape[0])

df_autres_1 = pd.concat([df_autres_0, df_préfixes, df_préfixes]).drop_duplicates(keep=False)
df_nc = df_autres_1[df_autres_1.Mot.str.islower()].copy()
print("nb mots noms communs:%d" % df_nc.shape[0])

df_autres_2 = pd.concat([df_autres_1, df_nc, df_nc]).drop_duplicates(keep=False)
df_np = df_autres_2[df_autres_2.Mot.str.istitle()].copy()
print("nb mots noms propres:%d" % df_np.shape[0])

df_autres_3 = pd.concat([df_autres_2, df_np, df_np]).drop_duplicates(keep=False)
df_sigles = df_autres_3[df_autres_3.Mot.str.isupper()].copy()
print("nb mots sigles:%d" % df_sigles.shape[0])

df_autres_4 = pd.concat([df_autres_3, df_sigles, df_sigles]).drop_duplicates(keep=False)
print("nb mots autres:%d" % df_autres_4.shape[0])

# mettre dans un ordre augmentant les probabilités de succès des noms composés
df_new = pd.concat([df_nc, df_sigles, df_np, df_composés, df_préfixes, df_sigles, df_autres_4], ignore_index=True).copy()
print("nb mots TOTAL:%d" % df_new.shape[0])

nb mots composés:159290
nb mots préfixes:424
nb mots noms communs:1292718
nb mots noms propres:28581
nb mots sigles:1971
nb mots autres:1059
nb mots TOTAL:1486014


In [17]:
df_autres_4.tail(5)

,Mot,Prononciation,H_aspiré,Type,Pré_valide,Warn_Code,Warn_Label,Plausible
1432947,xDSL,iks.de.ɛs‿ɛl,0.0,nom,1.0,-,-,?
1442217,éSwatini,e.swa.ti.ni,0.0,nom propre,1.0,-,-,?
1484012,€,ø.ʁo,0.0,symbole,1.0,-,-,?
1484013,℁,o bɔ̃ swɛ̃ də,0.0,préposition,1.0,-,-,?
1484014,∴,mɔʁ‿o.vaʃ,0.0,symbole_num=2,1.0,-,-,?


In [18]:
df2 = df_new
#df2=df2[df2.Mot.str.istitle()] # ne conserver que les mots commençant par une majuscule puis minuscules
#df2=df2[df2.Mot.str.islower()] # ne conserver que les mots entièrement en minuscules (i.e. noms communs)
#df2=df2[df2.Mot.str.isupper()] # ne conserver que les mots entièrement en majuscules (i.e. acronymes)#
#df2=df2[df2.Mot.str.startswith('x')] # ne conserver que les mots noms communs commençant par ...
#df2 = pd.concat([df2, df_composés, df_composés]).drop_duplicates(keep=False) # supprimer les mots composés
#df2 = df2[~df2.Mot.str.contains(' ')] # exclure les mots contenant un espace
#df2 = df2[~df2.Mot.str.contains('-')] # exclure les mots contenant un tiret

df2 = df2[~df2.Mot.str.contains('\.')] # exclure les mots contenant un point
df2 = df2[~df2.Mot.str.contains('/')] # exclure les mots contenant un slash (e.g. copier/coller)

df2.shape

(1485362, 8)

In [19]:
nb_samples=0
nb_infos_steps=10
nb_samples_steps = int(df2.shape[0]/nb_infos_steps)
nb_samples_ok=0
nb_samples_ko=0
nb_very_bad=0
nb_batch=0
verbose=False
indexes_to_forget = []
nb_composés = 0
nb_composés_directs_found = 0
nb_composés_indirects_found = 0

t0 = datetime.datetime.now()
for index, row in df2.iterrows():

    nb_samples += 1
    is_ok = False
    is_skipped = False
        
    if nb_samples >= nb_infos_steps and nb_samples % nb_samples_steps == 0:
      nb_batch += 1
      t1 = datetime.datetime.now()
      durée = t1 - t0
      print('batch:%d, nb_samples:%d, durée:%s' % (nb_batch, nb_samples, durée))
      t0 = t1

    mot = row['Mot']
    prononciation = row['Prononciation']
 
    is_composé = False
    is_composé_direct = False
    if ' ' in row['Mot'] or ',' in row['Mot'] or '-' in row['Mot']:
      nb_composés += 1     
      is_composé = True
        
    try:
      graphemes = row['Mot'].lower().replace(' ', '').replace('-','').replace(',','')
      phonemes = prononciation.replace('(','').replace(')','').replace('‿','').replace('.','').replace(' ','')
      is_ok = check_word_pronunciation(mot, phonemes, graphemes, False, False, False, False)
          
      if is_ok:
        if ' ' in row['Mot'] or ',' in row['Mot'] or '-' in row['Mot']:
          is_composé_direct = True
          nb_composés_directs_found += 1      
    
    except:
      is_ok = False
      is_skipped = True 
      nb_very_bad+=1
      # mauvaise lettre ou phoneme, pas la peine d'aller plus loi
      # passer au mot suivant

    # si la correspondance n'a pas été trouvé, tester si le mot est composé,
    # et si oui, tester la correspondance de chaque partie (aka sous-mot)
    if not is_ok and not is_skipped:    
      graphemes = row['Mot'].lower().replace(',','')
      phonemes = prononciation    
      separateurs = [' ', '-']
      for separateur in separateurs:
        if separateur in graphemes[1:-1] :
          mots_ = mot.split(separateur)
          # si le mot n'est pas un mot composé, l'oubler, et passer à la suite
          if len(mots_) <= 1:
            continue
          for mot_ in mots_:
            try:
              # récuperer le sous-mot identifiés ainsi que ses différentes prononciations possibles
              phonemess_ = df[df.Mot==mot_]['Prononciation'].values
              for phonemes_ in phonemess_:
                graphemes_ = mot_
                phonemes_ = phonemes_.replace('(','').replace(')','').replace('‿','').replace('.','').replace(' ','')
                is_ok_ = check_word_pronunciation(mot_, phonemes_, graphemes_, False, False, False, False)
                if is_ok_:
                    break                    
              if is_ok_ == False:
                is_ok = False                
              else:
                is_ok = True
                break
            except:
              is_ok = False
            
    if is_ok and is_composé:
        if not is_composé_direct:
            nb_composés_indirects_found += 1
    #if not is_ok and is_composé:
    #    print('word=%s \\\\%s\\\\'% (mot, prononciation))
        
    if is_ok:
      nb_samples_ok += 1
      df2.at[index, 'Plausible']='oui'
      indexes_to_forget.append(index)
    else:
      nb_samples_ko += 1
      if verbose:
        if row['Type'] != 'verbe_flexion':
          print('word=%s \\\\%s\\\\'% (mot, prononciation))
      df2.at[index, 'Plausible']='non'
        

t1 = datetime.datetime.now()
durée = t1 - t0
print('nb_samples:', nb_samples, ' durée de processing du dernier batch:', durée)
      
df2 = df2.drop(indexes_to_forget)

if nb_samples > 0:
  print('samples=%d, samples_ok=%d, samples_ko=%d, very_bad=%d, ok%%=%.2f' % \
        (nb_samples, nb_samples_ok, nb_samples_ko, nb_very_bad, \
         nb_samples_ok/nb_samples*100))

batch:1, nb_samples:148536, durée:0:00:39.005786
batch:2, nb_samples:297072, durée:0:00:37.977476
batch:3, nb_samples:445608, durée:0:00:40.337019
batch:4, nb_samples:594144, durée:0:00:38.807862
batch:5, nb_samples:742680, durée:0:00:38.149681
batch:6, nb_samples:891216, durée:0:00:39.266992
batch:7, nb_samples:1039752, durée:0:00:39.684357
batch:8, nb_samples:1188288, durée:0:00:38.967693
batch:9, nb_samples:1336824, durée:0:01:58.496086
batch:10, nb_samples:1485360, durée:0:04:25.546724
nb_samples: 1485362  durée de processing du dernier batch: 0:00:00.000503
samples=1485362, samples_ok=1471073, samples_ko=14289, very_bad=0, ok%=99.04


In [20]:
#nb_samples: 1292602  durée de processing du dernier batch: 0:00:00.000838
#samples=1292602, samples_ok=1279990, samples_ko=12612, very_bad=0 %ok=99.02

#samples=1485362, samples_ok=1471019, samples_ko=14343, very_bad=0, ok%=99.03
#nb_composés:160151
#nb_composés_directs_found:154583
#nb_composés_indirects_found:5216
#nb_composés_not_found:352

#samples=1485362, samples_ok=1468451, samples_ko=16911, very_bad=0, ok%=98.86
#nb_composés:160151
#nb_composés_directs_found:0
#nb_composés_indirects_found:157231
#nb_composés_not_found:2920

In [21]:
df2.shape

(14289, 8)

In [22]:
# 5 derniers mot dont la prononciation est détectée comme non plausible
df2[df2.Plausible=='non'].tail(20)

,Mot,Prononciation,H_aspiré,Type,Pré_valide,Warn_Code,Warn_Label,Plausible
1485872,Selk’nam,sɛlk.nam,0.0,nom,1.0,-,-,non
1485874,TADs,te.a.de,0.0,nom_flexion,1.0,-,-,non
1485960,URNs,y.ɛʁ.ɛn,0.0,nom_flexion,1.0,-,-,non
1485961,UpM,y.pe.em,0.0,nom,1.0,-,-,non
1485965,VO2max,ve.o.dø.maks,0.0,nom,1.0,-,-,non
1485972,Vosg’patt,voʒ.pat,0.0,nom,1.0,-,-,non
1485973,WEIs,ˈwaj ou ˈwɛj,0.0,nom_flexion,1.0,-,-,non
1485974,Wuchiaping’ien,wu.tʃja.piŋ.jɛ̃,0.0,nom,1.0,-,-,non
1485976,XPs,iks.pe,0.0,nom_flexion,1.0,-,-,non
1485978,Xi’an,ʃi.an,0.0,nom propre,1.0,-,-,non


In [23]:
# stockage des mots dot les prononciations sont non plausibles 
# dans un fichier CSV
#df2 = df2.drop(columns=['Plausible'])
df2.to_csv("correspondances_non_trouvées.csv", index=False, quotechar = '"')

In [24]:
df2[df2.Mot == 'patchworkisa']

,Mot,Prononciation,H_aspiré,Type,Pré_valide,Warn_Code,Warn_Label,Plausible


###Tests unitaire

In [25]:
# Mot et prononciation à tester unitairement

#graphemes_phonemes='brunch \\bʁœntʃ\\' 
graphemes_phonemes="patchworkisa \\patʃ.wœʁ.ki.za\\"

strings = graphemes_phonemes.split(' ')
mot=strings[0]
graphemes=mot.lower()
phonemes=strings[1]

# nettoyage des strings
#graphemes = graphemes.replace(' ','')
#graphemes = graphemes.replace('-','')
#graphemes = graphemes.replace("’",'')
phonemes = phonemes.replace(' ','')
phonemes = phonemes.replace('\\','')
phonemes = phonemes.replace('.','')
phonemes = phonemes.replace('‿','')
phonemes = phonemes.replace('(','')
phonemes = phonemes.replace(')','')

# test unitaire
check_word_pronunciation(mot, phonemes, graphemes, unit_test=True, verbose0=True, verbose1=True, verbose2=True)


process_graphemes(phonemes=\patʃwœʁkiza\, graphemes=[[patchworkisa]],     phoneme_index=0, grapheme_index=0,     phonemes_hist=[], graphemes_hist=[])
trying candidate current phoneme=p
process_graphemes(phonemes=\patʃwœʁkiza\, graphemes=[[patchworkisa]], phoneme_index=0, grapheme_index=0) current_phoneme:p
current_phoneme:p
phoneme \p\ matching graphemes [[p]] (last_graphemes:[[patchworkisa]])
trying2 current_phoneme \p\ and current_graphemes [[p]]

process_graphemes(phonemes=\patʃwœʁkiza\, graphemes=[[patchworkisa]],     phoneme_index=1, grapheme_index=1,     phonemes_hist=['p'], graphemes_hist=['p'])
trying candidate current phoneme=a
process_graphemes(phonemes=\patʃwœʁkiza\, graphemes=[[patchworkisa]], phoneme_index=1, grapheme_index=1) current_phoneme:a
current_phoneme:a
phoneme \a\ matching graphemes [[a]] (last_graphemes:[[atchworkisa]])
phoneme \a\ matching graphemes [[at]] (last_graphemes:[[atchworkisa]])
trying2 current_phoneme \a\ and current_graphemes [[a]]

process_graphem

True

## Statistiques

In [26]:
# Réaliser les comptages tottax statistiques sur les correspondances 
# précédemment comptées
dfp = make_stats()

In [27]:
# Afficher les statistiques d'un phonème en particulier (ex: \s\)
phoneme = 's'
#hex(ord('s'))

In [28]:
dfp[dfp.phonème == phoneme]

,phonème,graphème,occurences,pourcentage,exemple_1,exemple_2,exemple_3
248,s,c’,177,0.00,c’,c’est,c’que
249,s,s’,2968,0.00,s’abader,s’abaisser,s’abaisser
250,s,ç’,1,0.00,ç’,,
251,s,c,81125,0.12,abaciste,abacistes,abandogiciel
252,s,ç,10319,0.01,accoinçon,accoinçons,acquiesçant
253,s,s,286180,0.45,aaronisme,aaronismes,aasax
254,s,sc,5171,0.00,abscise,abscises,abscision
255,s,th,132,0.00,chrestomathie,chrestomathies,classpath
256,s,x,460,0.00,antisoixantehuitard,antisoixantehuitards,auxerrois
257,s,cc,50,0.00,accensa,accensai,accensaient


## Bilan des correspondances non trouvées

* Normalement, aucune ligne ne devrait être affichée (si noms communs, noms propres et sigles analysés)

In [29]:
pd.set_option('display.max_rows', 100)
dfp[dfp.occurences == 0]

,phonème,graphème,occurences,pourcentage,exemple_1,exemple_2,exemple_3
378,ɥ,’hu,0,0.0,,,
576,œ̃,Hun,0,0.0,,,
577,œ̃,Huns,0,0.0,,,
597,œ,ogl,0,0.0,,,
611,ø,hœ,0,0.0,,,
619,ø,euent,0,0.0,,,
807,ɔ̃,ump,0,0.0,,,
835,ɑ̃,aën,0,0.0,,,
904,a,acts,0,0.0,,,
940,ɔɛ,œ,0,0.0,,,


In [30]:
#from google.colab import files
#files.download('phonemes2graphemes.csv')
#files.download('correspondances_non_trouvées.csv')